## Load Environment Variables

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
cohere_api_key= os.getenv('COHERE_API_KEY')
google_api_key= os.getenv('GOOGLE_API_KEY')

## Load Dependencies

In [29]:
#Load Chain Dependencies
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate
)
from langchain.chains import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings, HuggingFaceBgeEmbeddings, GooglePalmEmbeddings, HuggingFaceEmbeddings, GPT4AllEmbeddings

# Load the LLMS
from langchain.llms import LlamaCpp, Cohere, GooglePalm
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


#Loader Dependencies
from bs4 import BeautifulSoup as Soup

#Load the loaders
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader

#Load the tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

#Load the agent
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

## Declare Variables

In [30]:
google_palm_llm = GooglePalm()
cohere_llm = ChatLiteLLM(model="command-nightly")

In [31]:

tenders_pdf_loader = PyPDFDirectoryLoader("../../data/")

In [32]:

#loader = DirectoryLoader('../', glob="**/*.md", use_multithreading=True)
#txt_loader = DirectoryLoader("../../data/talentlms/", glob="**/*.txt", use_multithreading=True, loader_cls=TextLoader)
#xlsx_loader = DirectoryLoader("../../data/talentlms/", glob="**/*.xlsx", use_multithreading=True, loader_cls=UnstructuredExcelLoader)
pdf_loader = DirectoryLoader("../../data/tenders/", glob="**/*.pdf", use_multithreading=True, loader_cls=PyPDFDirectoryLoader)
#docx_loader = DirectoryLoader("../../data/talentlms/", glob="**/*.docx", use_multithreading=True, loader_cls=UnstructuredWordDocumentLoader)

In [33]:
url = ""
talent_lms_recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=200, extractor=lambda x: Soup(x, "html.parser").text)

In [34]:
loaders = [tenders_pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 149


In [35]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [36]:
model_path = "../../../../llms/the_bloke/tinyllama-1.1b-intermediate-step-715k-1.5t.Q4_K_M.gguf"

In [37]:
n_gpu_layers = 250  # Change this value based on your model and your GPU VRAM pool.
n_batch = 4096  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# Make sure the model path is correct for your system!
llamacpp_llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=8196,
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 201 tensors from ../../../../llms/the_bloke/tinyllama-1.1b-intermediate-step-715k-1.5t.Q4_K_M.gguf (version unknown)
llama_model_loader: - tensor    0:                    output.weight q6_K     [  2048, 32000,     1,     1 ]
llama_model_loader: - tensor    1:                token_embd.weight q4_K     [  2048, 32000,     1,     1 ]
llama_model_loader: - tensor    2:           blk.0.attn_norm.weight f32      [  2048,     1,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_K     [  2048,  2048,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_K     [  2048,   256,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q6_K     [  2048,   256,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q4_K     [  2048,  2048,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_norm.w

In [38]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = GooglePalmEmbeddings()

In [39]:
db = Chroma.from_documents(texts, embeddings, collection_name="greg_talentlms")

In [40]:
greg_docs_chain = RetrievalQA.from_chain_type(
    llm=google_palm_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [41]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()
#Python REPL Class
python_repl = PythonREPL()

In [42]:
tools = [
    Tool(
        name="Greg Shield - Tenders",
        func=greg_docs_chain.run,
        description="""Useful for when you need to answer questions about tenders. This is specifically for tenders in training. These tenders are usually by state-owned companies but they can be any entity interested in digital skills.
        This chain will be usefully for document authoring and strategic planning for which tenders Deviare should apply for.
        """,
    ),
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [43]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, google_palm_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

In [44]:
prefix = """Take a deep breath and solve the following question step-by-step.

You are Greg. As Greg, you are a tenders expert who will help Deviare author replies/proposals to RFQs/RFPs. As a business analyst at a digital transformation company, I need your assistance in understanding the context and business requirements for a request for proposal (RFP) or request for quotation (RFQ). I will provide content that contains the RFP/RFQ information. Please use the information below and provide me with the following details to help me better grasp the nature of the requirement. List where I can find each requirement:

1. Due Date: Please identify the due date for the RFP/RFQ.

2. Department: Determine which department the RFP/RFQ belongs to.

3. Brief Description: Provide a brief description of the bid outlined in the document.

4. Opening Date: Identify the date when the RFP/RFQ will be opened.

5. Closing Date: Determine the date when the RFP/RFQ will be closed.

6. Briefing Session: Check if there is a briefing session scheduled for the RFP/RFQ and provide the date if available.

7. Compulsory Briefing: Indicate whether the briefing session is compulsory for participants.

8. Contact Person: Identify the contact person for further inquiries regarding the RFP/RFQ.

9. Location: Determine the location where goods, works, or services are required as mentioned in the document.

10. Proposal Requirements: Extract and summarize the specific proposal requirements that need to be addressed.

11: Evaluation Criteria: Please provide the important evaluation criteria on which the proposals will be assessed. Please be list these and try to point them out clearly as this is what we will be assessed on. use an unordered list to render each

Functionality Criteria: Please provide and list all relevant functionality criteria you find on the RFQ3

Scope of Work

I will provide content for your review shortly"""
suffix = """
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [45]:
llm_chain = LLMChain(llm=google_palm_llm, prompt=prompt)

In [46]:
tool_names = [tool.name for tool in tools]
v_agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [47]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=v_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [51]:
agent_executor.run("""Help me qualify the City of Joburg Tender and how we can solve for it. Write a short business proposal with a executive summary, programme overview and implementation plan""")



> Entering new AgentExecutor chain...
Thought: I need to understand the context and business requirements for the City of Joburg Tender
Action: Greg Shield - Tenders
Action Input: City of Joburg Tender
Observation: APPOINTMENT OF A PANEL OF SERVICE PROVIDERS FOR TRAINING SERVICES FOR THE CITY OF JOHANNESBURG FOR A PERIOD OF THREE (3) YEARS AS AND WHEN REQUIRED.
Thought:I need to identify the due date for the City of Joburg Tender
Action: Greg Shield - Tenders
Action Input: City of Joburg Tender
Observation: BID NO: COJ/GCSS001/23 -24
Thought:I need to determine the date when the City of Joburg Tender will be opened
Action: Greg Shield - Tenders
Action Input: City of Joburg Tender
Observation: The tender is for the appointment of a panel of service providers for training services for the City of Johannesburg for a period of three (3) years as and when required.
Thought:I need to determine the date when the City of Joburg Tender will be closed
Action: Greg Shield - Tenders
Action Input

'**Question:** Help me qualify the City of Joburg Tender and how we can solve for it. Write a short business proposal with a executive summary, programme overview and implementation plan'